In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import requests
import bs4
from bs4 import BeautifulSoup
from selenium.common.exceptions import *
from selenium import webdriver
from tqdm.notebook import tqdm
%matplotlib inline

In [3]:
def get_arxiv_total_entries(year, category):
    y = int(str(year)[-2:])
    origin_url = r"https://arxiv.org/list/{}/{}?skip=0".format(category, y)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
    r = requests.get(origin_url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")
    text = soup.find(name="div", attrs={"id": "dlpage"})
    total_entries = text.find(name="small")
    total_entries = int(total_entries.contents[0].split()[3])
    return total_entries

In [4]:
def get_arxiv_paper_list(total_entries, year, category):
    y = int(str(year)[-2:])
    driver_path = r"C:\Users\YangWang\Desktop\machineLearning\indiaNewsClassification\chromedriver.exe"
    driver = webdriver.Chrome(driver_path)
    time.sleep(2)
    paper_list = []

    for num in range(int(total_entries/2000)+1):
        url = r"https://arxiv.org/list/{}/{}?skip={}&show=2000".format(category, y, num*2000)
        driver.get(url)
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        content = soup.find(name="div", attrs={"id": "content"})
        dl = content.find(name="dl")

        dd = dl.find_all(name="dd")
        dt = dl.find_all(name="dt")
        for e1, e2 in zip(dd, dt):
            if e1.find(name="div", attrs={"class": "list-title mathjax"}) is not None:
                title = e1.find(name="div", attrs={"class": "list-title mathjax"}).text
            else:
                title = None
            if e1.find(name="span", attrs={"class": "primary-subject"}) is not None:
                subject = e1.find(name="span", attrs={"class": "primary-subject"}).string
            else:
                subject = None
            if e2.find(name="span", attrs={"class": "list-identifier"}) is not None:
                link = e2.find(name="span", attrs={"class": "list-identifier"}).find(name="a").get("href")
            else:
                link = None
            year = str(year)
            paper_list.append([title, subject, link, year])

    driver.quit()
    
    return paper_list

In [5]:
def dataframe_preprocessing(paper_list):
    df = pd.DataFrame(paper_list)
    df.columns = ["title", "subject", "link", "year"]
    df.reset_index(drop=True)

    df["title"] = df["title"].map(lambda x: x.split(":")[1])
    df["title"] = df["title"].map(lambda x: x.split("\n")[0])
    df["link"] = df["link"].map(lambda x: "https://arxiv.org" + x if x is not None else None)
    df["year"] = df["year"].map(int)
    
    return df

In [6]:
def get_arxiv_dataframe(queue, category, start_year=2013, end_year=2020):
    """
    category:
        "cs": computer science
        "math": mathematics
        "stat": statistics
        "eess": electrical engineering and systems science
        "q-fin": quantitative finance
    """
    final_df = pd.DataFrame()
    for year in tqdm(range(start_year, end_year+1)):
        total_entries = get_arxiv_total_entries(year, category)
        paper_list = get_arxiv_paper_list(total_entries, year, category)
        df = dataframe_preprocessing(paper_list)
        final_df = pd.concat([final_df, df])
        
        final_df = final_df.drop_duplicates()
        final_df.dropna(inplace=True)
    queue.put(final_df)

In [7]:
import threading
import queue

subject = ["cs", "math", "stat", "eess", "q-fin"]
df = {name: pd.DataFrame() for name in subject}
paper_df = pd.DataFrame()

threads = []
my_queue = queue.Queue()
for name, df in df.items():
    threads.append(threading.Thread(target = get_arxiv_dataframe, args = (my_queue, name)))
    #df = get_arxiv_dataframe(category = name, start_year=2010, end_year=2020)
    #paper_df = pd.concat([paper_df, df])
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
while my_queue.qsize() > 0:
    df = my_queue.get()
    paper_df = pd.concat([paper_df, df])
paper_df.reset_index(drop=True)

,title,subject,link,year
0,"Trading networks, abnormal motifs and stock m...",Trading and Market Microstructure (q-fin.TR),https://arxiv.org/abs/1301.0007,2013
1,The effect of debt on corporate profitability,General Finance (q-fin.GN),https://arxiv.org/abs/1301.0072,2013
2,On Reduced Form Intensity-based Model with Tr...,Computational Finance (q-fin.CP),https://arxiv.org/abs/1301.0109,2013
3,On Infectious Model for Dependent Defaults,Risk Management (q-fin.RM),https://arxiv.org/abs/1301.0186,2013
4,Utility maximization with current utility on ...,Portfolio Management (q-fin.PM),https://arxiv.org/abs/1301.0280,2013
...,...,...,...,...
569470,Path integral representation for inverse thir...,High Energy Physics - Theory (hep-th),https://arxiv.org/abs/2004.04958,2020
569471,Bounding the Mim-Width of Hereditary Graph Cl...,Data Structures and Algorithms (cs.DS),https://arxiv.org/abs/2004.05018,2020
569472,Colouring (sP1+P5)(sP_1+P_5)-Free Graphs,Data Structures and Algorithms (cs.DS),https://arxiv.org/abs/2004.05022,2020
569473,Functional Equations and Separation of Variab...,High Energy Physics - Theory (hep-th),https://arxiv.org/abs/2004.05071,2020


In [8]:
paper_df.to_csv("arxiv_paper.csv", index=False)